Есть данные, записыванные в разные файлы по завершенным урокам в учебном приложении. Нужно исследовать их и ответить на ряд вопросов.

Файлы: 

1. assessments.csv — этот файл содержит информацию об оценках в тесте. Обычно каждый предмет в семестре включает ряд тестов с оценками, за которыми следует заключительный экзаменационный тест (экзамен):
 - code_module — идентификационный код предмета.

 - code_presentation — семестр (Идентификационный код).

 - id_assessment — тест (Идентификационный номер ассессмента).

 - assessment_type — тип теста. Существуют три типа оценивания: оценка преподавателя (TMA), компьютерная оценка (СМА), экзамен по курсу (Exam).

 - date — информация об окончательной дате сдачи теста. Рассчитывается как количество дней с момента начала семестра. Дата начала семестра имеет номер 0 (ноль).

 - weight — вес теста в % в оценке за курс. Обычно экзамены рассматриваются отдельно и имеют вес 100%; сумма всех остальных оценок составляет 100%.


2. courses.csv — файл содержит список предметов по семестрам:
 - code_module — предмет (идентификационный код).

 - code_presentation — семестр (идентификационный код).

 - module_presentation_length — продолжительность семестра в днях.


3. studentAssessment.csv — этот файл содержит результаты тестов студентов. Если учащийся не отправляет работу на оценку, результат не записывается в таблицу:
 - id_assessment — тест (идентификационный номер).

 - id_student — идентификационный номер студента.

 - date_submitted — дата сдачи теста студентом, измеряемая как количество дней с начала семестра.

 - is_banked — факт перезачета теста с прошлого семестра (иногда курсы перезачитывают студентам, вернувшимся из академического отпуска).

 - score — оценка учащегося в этом тесте. Диапазон составляет от 0 до 100. Оценка ниже 40 неудачная/неуспешная сдача теста.


4. studentRegistration.csv — этот файл содержит информацию о времени, когда студент зарегистрировался для прохождения курса в семестре:
 - code_module — предмет (идентификационный код).

 - code_presentation — семестр (идентификационный код)

 - id_student — идентификационный номер студента.

 - date_registration — дата регистрации студента. Это количество дней, измеренное от начала семестра (например, отрицательное значение -30 означает, что студент зарегистрировался на прохождение курса за 30 дней до его начала).

 - date_unregistration — дата отмены регистрации студента с предмета. У студентов, окончивших курс, это поле остается пустым.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
scores = pd.read_csv ('assessments.csv')
courses = pd.read_csv ('courses.csv')
student_scores = pd.read_csv ('studentAssessment.csv')
student_reg = pd.read_csv ('studentRegistration.csv')

Сначала нужно провести предварительное исследование данных и определить что является учебным курсом. Я определяю учебный курс как совокупность учебных предметов, заканчивающихся экзаменами. Учебный курс длится определенное время, необходимое на его освоение.

In [3]:
scores.nunique () 

code_module            7
code_presentation      4
id_assessment        206
assessment_type        3
date                  74
weight                24
dtype: int64

In [4]:
scores.value_counts ('assessment_type')

assessment_type
TMA     106
CMA      76
Exam     24
dtype: int64

В датафрейме с предметами есть 7 уникальных идентификационных кодов модулей (предметов?). 
При этом всего есть 206 тестов, из которых лишь 24 - экзамены. 
Проверим, заканчиваются ли каждый модуль экзаменом. 

In [5]:
scores.query ('assessment_type == "Exam"')

,code_module,code_presentation,id_assessment,assessment_type,date,weight
5,AAA,2013J,1757,Exam,NaN,100.0
11,AAA,2014J,1763,Exam,NaN,100.0
23,BBB,2013B,14990,Exam,NaN,100.0
35,BBB,2013J,15002,Exam,NaN,100.0
47,BBB,2014B,15014,Exam,NaN,100.0
53,BBB,2014J,15025,Exam,NaN,100.0
62,CCC,2014B,24290,Exam,NaN,100.0
63,CCC,2014B,40087,Exam,NaN,100.0
72,CCC,2014J,24299,Exam,NaN,100.0
73,CCC,2014J,40088,Exam,NaN,100.0


Действительно, каждый модуль включат в себя минимум 2 экзамена и длится минимум 2 семестра. Это наводит на мысль, что код модуля - это уникальный идентификатор курса, а каждый из предметов в рамках курса проходится в определённом семестре и заканчиватся экзаменом. Проверим наше предположение распределением студентов по модулям.  

In [6]:
student_reg. groupby ('code_module'). agg ({'id_student': 'count'})

,id_student
code_module,
AAA,748
BBB,7909
CCC,4434
DDD,6272
EEE,2934
FFF,7762
GGG,2534


Мы видим, что на каждый из модулей зарегистрировалось большое количество студентов. 
Вряд ли бы такое количество людей зарегистрировалось на отдельные предметы вне рамок курсов.
Всё вышесказанное позволяет заключить, что модули - это курсы.

Далее нужно определить количество студентов успешно закончивших только один курс (успешная сдача — это зачёт по курсу на экзамене).

Для этого сначала надо объединить датафреймы с оценками студентов и общей информацией о тестах для того, чтобы совместить  идентификатор студента с идентификатором экзамена и идентификатором курса. 

In [7]:
total_scores = student_scores.merge (scores, on = 'id_assessment')
total_scores. head (5)

# Объединяем датафреймы по общей колонке и проверяем корректность результата 

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight
0,1752,11391,18,0,78.0,AAA,2013J,TMA,19.0,10.0
1,1752,28400,22,0,70.0,AAA,2013J,TMA,19.0,10.0
2,1752,31604,17,0,72.0,AAA,2013J,TMA,19.0,10.0
3,1752,32885,26,0,69.0,AAA,2013J,TMA,19.0,10.0
4,1752,38053,19,0,79.0,AAA,2013J,TMA,19.0,10.0


Далее нужно отфильтровать студентов успешно сдавших экзамен, т.е. сдавших его на 40 баллов и выше.

In [8]:
score_success_ex = total_scores. query ('assessment_type == "Exam" & score >=40')

# Создаём новый датафрейм с отфильтрованными студентами по условию

Теперь остался финальный шаг - определить количество студентов успешно закончивших только 1 курс

In [9]:
score_success_ex. groupby ('id_student', as_index = False) \
    .agg ({'code_module': 'count'}) \
    .query ('code_module == 1'). id_student. count () 

# Для этого мы группируем датафрейм по каждому студенту; считаем количество курсов, которые он закончил; 
# отбираем закончивших один курс и считаем количество таких студентов

3802

В итоге, 3802 студента успешно закончили только один курс

Следующая задача - выявить курсы и экзамены в рамках курса, которые обладают самой низкой и самой высокой завершаемостью

In [10]:
score_ex = total_scores. query ('assessment_type == "Exam"')
# Создаём датафрейм со всеми студентами, которые сдавали экзамены

In [11]:
sc_ex = score_ex. groupby (['id_assessment', 'code_module'], as_index = False) \
    .agg ({'id_student': 'count'}) \
    .rename (columns = {'id_student': 'total_attempts'})
sc_ex

# Создаём датафрейм с группировкой по коду экзамена и курса, считаем количество студентов, пытавшихся сдать их;
# переименовываем колонку со студентами в "количество попыток"

,id_assessment,code_module,total_attempts
0,24290,CCC,747
1,24299,CCC,1168
2,25340,DDD,602
3,25354,DDD,968
4,25361,DDD,524
5,25368,DDD,950


In [12]:
sc_sucs_ex = score_success_ex. groupby (['id_assessment', 'code_module'], as_index = False) \
    .agg ({'id_student': 'count'}). rename (columns = {'id_student': 'succesful_attempts'})
sc_sucs_ex

# Делаем то же самое, что и в предыдущем случае, но считаем количество успешных попыток сдачи экзаменов из соответствующего датафрейма
# переименовываем колонку со студентами в "успешные попытки"

,id_assessment,code_module,succesful_attempts
0,24290,CCC,664
1,24299,CCC,1019
2,25340,DDD,504
3,25354,DDD,878
4,25361,DDD,485
5,25368,DDD,842


In [13]:
sc_sucs_ex ['total_attempts'] = sc_ex.total_attempts

# Добавляем колонку с общим числом попыток сдать экзамены в агрегированный датафрейм с успешным числом соответствующих попыток

In [14]:
sc_sucs_ex ['completion_score'] = sc_sucs_ex.succesful_attempts / sc_sucs_ex.total_attempts
round (sc_sucs_ex. sort_values (['code_module', 'completion_score'], ascending = False), 2) 

# Создаём колонку с метрикой завершаемости, которая считается как количество успешных попыток сдать экзамен, деленное на количество всех попыток
# Сортируем датафрейм по убыванию завершаемости экзаменов внутри каждого курса и округляем результат до сотых

,id_assessment,code_module,succesful_attempts,total_attempts,completion_score
4,25361,DDD,485,524,0.93
3,25354,DDD,878,968,0.91
5,25368,DDD,842,950,0.89
2,25340,DDD,504,602,0.84
0,24290,CCC,664,747,0.89
1,24299,CCC,1019,1168,0.87


In [15]:
round (sc_sucs_ex. groupby ('code_module'). agg ({'completion_score': ['mean', 'median']}), 2)

# Считаем среднее и медиану завершаемости курсов

completion_score       
                        mean median
code_module                        
CCC                     0.88   0.88
DDD                     0.89   0.90

Итак, мы видим, что в среднем курс DDD обладает более высокой завершаемостью, чем курс ССС (правда, различия невелики).

Если говорить об экзаменах в рамках этих курсов, то:

1) экзамен под кодом 25361 в курсе DDD обладает самой высокой завершаемостью в целом;

2) экзамен под кодом 25340 в курсе DDD обладает самой низкой завершаемостью в целом;

3) экзамен под кодом 24290 в курсе ССС обладает самой высокой завершаемостью в этом курсе;

4) экзамен под кодом 24299 в курсе ССС обладает самой низкой завершаемостью в этом курсе.

Следующая задача - определить средний срок сдачи экзаменов по каждому предмету

In [16]:
round (score_success_ex. groupby (['id_assessment', 'code_module'], as_index = False) \
       .agg ({'date_submitted': 'mean'}) \
       .sort_values ('date_submitted'), 0)

# Для этого мы взяли отфильтрованный датафрейм со студентами, успешно сдавшими экзамен, и сгруппировали его по коду экзамена и курса
# Далее мы агрегировали среднее от даты последних сдач экзаменов и отсортировали результат по возрастанию

,id_assessment,code_module,date_submitted
2,25340,DDD,230.0
0,24290,CCC,232.0
4,25361,DDD,235.0
3,25354,DDD,240.0
5,25368,DDD,243.0
1,24299,CCC,244.0


Следующая задача - выявить самые популярные предметы (ТОП-3) по количеству регистраций на них и ТОП-3 предметов с самым большим оттоком. 

Для этого сначала надо объединить датафреймы с оценками студентов и регистрацией студентов для того, чтобы совместить идентификаторы студента и курса с датами регистрации и отмены регистрации студентов.

In [17]:
reg_df = student_scores.merge (student_reg, on = 'id_student')
reg_df
# Объединяем датафреймы по общей колонке и проверяем корректность результата 

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,date_registration,date_unregistration
0,1752,11391,18,0,78.0,AAA,2013J,-159.0,NaN
1,1753,11391,53,0,85.0,AAA,2013J,-159.0,NaN
2,1754,11391,115,0,80.0,AAA,2013J,-159.0,NaN
3,1755,11391,164,0,85.0,AAA,2013J,-159.0,NaN
4,1756,11391,212,0,82.0,AAA,2013J,-159.0,NaN
...,...,...,...,...,...,...,...,...,...
207314,37439,573320,227,0,80.0,GGG,2014J,-4.0,NaN
207315,37440,573320,227,0,100.0,GGG,2014J,-4.0,NaN
207316,37441,573320,227,0,100.0,GGG,2014J,-4.0,NaN
207317,37442,573320,227,0,20.0,GGG,2014J,-4.0,NaN


In [18]:
reg_df. groupby (['id_assessment', 'code_module']) \
    .agg ({'date_registration': 'count'}) \
    .sort_values ('date_registration', ascending = False). head (3) 

# Далее мы группируем предыдущий датафрейм по кодам экзаменов и курсов, считаем количество регистраций на них
# Затем сортируем результат по возрастанию и выводим ТОП-3 предметов по количеству регистраций

,,date_registration
id_assessment,code_module,
24295,CCC,2056
34873,FFF,2021
34899,FFF,1998


Для того, чтобы найти предметы с самым большим оттоком нужно отфильтровать всех студентов, не отменивших регистрацию и окончивших курс

In [19]:
reg_dropna = reg_df. dropna ()

# Создаём датафрейм с удалёнными пустыми значениями в поле отмены регистрации

In [20]:
reg_dropna. groupby (['id_assessment', 'code_module']) \
    .agg ({'date_unregistration': 'count'}) \
    .sort_values ('date_unregistration', ascending = False). head (3) 

# Далее мы группируем предыдущий датафрейм по кодам экзаменов и курсов, считаем количество отменённых регистраций
# Затем сортируем результат по возрастанию и выводим ТОП-3 предметов по оттоку

,,date_unregistration
id_assessment,code_module,
24295,CCC,710
34899,FFF,508
24286,CCC,470


Следующая задача - выявить семестр с самой низкой завершаемостью курсов и самыми долгими средними сроками сдачи курсов

In [21]:
sem_sc_ex = score_ex. groupby ('code_presentation', as_index = False). agg ({'id_student': 'count'})
sem_sc_ex  = sem_sc_ex . rename (columns = {'id_student': 'total_attempts'})
sem_sc_ex

# Создаём датафрейм с группировкой по номеру семестра, считаем количество студентов, пытавшихся сдать экзамены в этом семестре;
# переименовываем колонку со студентами в "количество попыток"

,code_presentation,total_attempts
0,2013B,602
1,2013J,968
2,2014B,1271
3,2014J,2118


In [22]:
sem_sc_sucs_ex = score_success_ex. groupby ('code_presentation', as_index = False). agg ({'id_student': 'count'})
sem_sc_sucs_ex = sem_sc_sucs_ex. rename (columns = {'id_student': 'succesful_attempts'})
sem_sc_sucs_ex

# Создаём датафрейм с группировкой по номеру семестра, считаем количество студентов, успешно сдавших экзамены в этом семестре;
# переименовываем колонку со студентами в "успешные попытки"

,code_presentation,succesful_attempts
0,2013B,504
1,2013J,878
2,2014B,1149
3,2014J,1861


In [23]:
sem_sc_sucs_ex ['total_attempts'] = sem_sc_ex.total_attempts
sem_sc_sucs_ex

# Добавляем колонку с общим числом попыток сдать экзамены в агрегированный датафрейм с успешным числом соответствующих попыток

,code_presentation,succesful_attempts,total_attempts
0,2013B,504,602
1,2013J,878,968
2,2014B,1149,1271
3,2014J,1861,2118


In [24]:
sem_sc_sucs_ex ['completion_score'] = sem_sc_sucs_ex.succesful_attempts / sem_sc_ex.total_attempts 
sem_sc_sucs_ex = round (sem_sc_sucs_ex. sort_values ('completion_score'), 2) 
sem_sc_sucs_ex

# Создаём колонку с метрикой завершаемости по семестрам
# Сортируем датафрейм по возрастанию завершаемости экзаменов по семестрам и округляем результат до сотых

,code_presentation,succesful_attempts,total_attempts,completion_score
0,2013B,504,602,0.84
3,2014J,1861,2118,0.88
2,2014B,1149,1271,0.90
1,2013J,878,968,0.91


In [25]:
sem_dt_ex = round (score_success_ex. groupby ('code_presentation', as_index = False) \
       .agg ({'date_submitted': 'mean'}) \
       .sort_values ('date_submitted'), 0)
sem_dt_ex

# Берём отфильтрованный датафрейм со студентами, успешно сдавшими экзамен, группируем его по номеру семестра
# Далее считаем среднее от даты последних сдач экзаменов и сортируем результат по возрастанию

,code_presentation,date_submitted
0,2013B,230.0
2,2014B,233.0
1,2013J,240.0
3,2014J,244.0


In [26]:
sem_ex = sem_sc_sucs_ex.merge (sem_dt_ex, on = 'code_presentation')
sem_ex

# Объединяем предыдущие датафреймы для определения семестра с самой низкой завершаемостью курсов и самыми долгими средними сроками сдачи курсов

,code_presentation,succesful_attempts,total_attempts,completion_score,date_submitted
0,2013B,504,602,0.84,230.0
1,2014J,1861,2118,0.88,244.0
2,2014B,1149,1271,0.90,233.0
3,2013J,878,968,0.91,240.0


В итоге, 2013B - семестр с самой низкой завершаемостью курсов и самыми долгими средними сроками сдачи курсов

Следующая задача - провести адаптированный RFM-анализ аудитории.
В адаптированной кластеризации я выбрал следующие метрики: R - среднее время сдачи одного экзамена, F - завершаемость курсов, M - среднее количество баллов, получаемое за экзамен.

In [27]:
sc_ex = score_ex.drop (columns= ['is_banked', 'date', 'weight'])
sc_ex.head (5)

# Удаляю из датафрейма с оценками студентов ненужные колонки и записываю это в новый датафрейм с более удобным названием

,id_assessment,id_student,date_submitted,score,code_module,code_presentation,assessment_type
52923,24290,558914,230,32.0,CCC,2014B,Exam
52924,24290,559706,234,78.0,CCC,2014B,Exam
52925,24290,559770,230,54.0,CCC,2014B,Exam
52926,24290,560114,230,64.0,CCC,2014B,Exam
52927,24290,560311,234,100.0,CCC,2014B,Exam


In [28]:
r_df = sc_ex. groupby ('id_student', as_index = False). agg ({'date_submitted': 'mean'})
r_df = r_df.rename (columns = {'date_submitted': 'avg_time'})
r_df. head (5)

# Считаю среднее время сдачи экзаменов студентами и записываю результат в датафрейм с R-метрикой

,id_student,avg_time
0,23698,243.0
1,24213,236.0
2,27116,243.0
3,28046,237.0
4,28787,243.0


На мой взгляд, завершаемость курса студентами можно считать как отношение успешных попыток конкретного студента сдать курс к количеству таких попыток. Задача осложняется тем, что студент может проходить сразу несколько курсов. Поэтому нормируем попытки сдачи экзамена.

In [29]:
def f(row):
 if row['score'] < 40:
     val = 0
 else:
     val = 1
 return val

sc_ex ['f_coef'] = sc_ex.apply (f, axis = 1) 
sc_ex.head(5)

# определяем функцию, которая присваивает единицу пользователю за удачную сдачу экзамена и ноль - за неудачную
# создаём колонку датафрейма, реализующую эту функцию

,id_assessment,id_student,date_submitted,score,code_module,code_presentation,assessment_type,f_coef
52923,24290,558914,230,32.0,CCC,2014B,Exam,0
52924,24290,559706,234,78.0,CCC,2014B,Exam,1
52925,24290,559770,230,54.0,CCC,2014B,Exam,1
52926,24290,560114,230,64.0,CCC,2014B,Exam,1
52927,24290,560311,234,100.0,CCC,2014B,Exam,1


In [30]:
f_coef_c = sc_ex. groupby ('id_student', as_index = False). agg ({'f_coef': 'count'})
f_coef_c = f_coef_c.rename (columns = {'f_coef': 'attempts_number'})
f_coef_c.head(5)

# считаем количество попыток сдать экзамены, независимо от курсов

,id_student,attempts_number
0,23698,1
1,24213,1
2,27116,1
3,28046,1
4,28787,1


In [31]:
f_coef_sum = sc_ex. groupby ('id_student', as_index = False). agg ({'f_coef': 'sum'})  
f_coef_sum = f_coef_sum.rename (columns = {'f_coef': 'f_coef_sum'})
f_coef_sum.head(5)

# считаем количество удачных попыток сдать экзамены, независимо от курсов

,id_student,f_coef_sum
0,23698,1
1,24213,1
2,27116,1
3,28046,1
4,28787,1


In [32]:
f_df = f_coef_sum. merge (f_coef_c, on = 'id_student')
f_df ['completion_score'] = f_df.f_coef_sum / f_df.attempts_number
f_df.head(5)

# объединяем предыдущие датафреймы в новый и создаем колонку с нормированной метрикой завершаемости

,id_student,f_coef_sum,attempts_number,completion_score
0,23698,1,1,1.0
1,24213,1,1,1.0
2,27116,1,1,1.0
3,28046,1,1,1.0
4,28787,1,1,1.0


In [33]:
f_df. value_counts ('completion_score')

# мы видим, что нормированная метрика завершаемости принимает 3 значения
# можно преположить, что самое малочисленное из них (0.5) относится к людям, которые записались на 2 курса, но успешно сдали лишь один

completion_score
1.0    4069
0.0     536
0.5      28
dtype: int64

Проверим эту гипотезу

In [34]:
reg_df_m = reg_df.merge (f_df, on = 'id_student')
reg_df_m = reg_df_m.drop (['f_coef_sum', 'attempts_number'], axis = 1)
reg_df_m.head(5)

# объединяем датафреймы с регистрациями и метрикой завершаемости, чтобы у нас были код студентов, теста и коэффицент метрики в одном датафрейме

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,date_registration,date_unregistration,completion_score
0,14996,523857,11,0,72.0,BBB,2013J,-100.0,NaN,1.0
1,14996,523857,11,0,72.0,DDD,2014B,-110.0,NaN,1.0
2,14997,523857,45,0,73.0,BBB,2013J,-100.0,NaN,1.0
3,14997,523857,45,0,73.0,DDD,2014B,-110.0,NaN,1.0
4,14998,523857,94,0,68.0,BBB,2013J,-100.0,NaN,1.0


In [35]:
reg_df_compl_sc_half = reg_df_m. query ('completion_score == 0.5')
reg_df_compl_sc_half. head(5)
# отсеиваем студентов с нужным значением метрики в новый датафрейм

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,date_registration,date_unregistration,completion_score
2686,24282,465843,39,0,32.0,CCC,2014B,-219.0,NaN,0.5
2687,24282,465843,39,0,32.0,DDD,2013B,-30.0,NaN,0.5
2688,24283,465843,109,0,79.0,CCC,2014B,-219.0,NaN,0.5
2689,24283,465843,109,0,79.0,DDD,2013B,-30.0,NaN,0.5
2690,24284,465843,151,0,56.0,CCC,2014B,-219.0,NaN,0.5


In [36]:
reg_df_compl_sc_half.id_student. nunique ()

# смотрим количество уникальных пользователей (28)

28

In [37]:
reg_df_compl_sc_half. groupby (['id_student', 'code_module'], as_index = False). agg ({'id_assessment' : 'count'}). nunique ()

# группируем датафрейм по пользователям и курсам, смотрим количество курсов и сравниваем количество уникальных пользователей до и после группировки

id_student       28
code_module       2
id_assessment    13
dtype: int64

Таким образом, мы увидели, что все 28 студентов действительно проходили два курса

In [38]:
rf_df = r_df. merge (f_df, on = 'id_student'). drop (columns = ['f_coef_sum', 'attempts_number'])
rf_df.head (5)

# объединяем датафреймы с R и F метриками в один и удаляем лишние колонки

,id_student,avg_time,completion_score
0,23698,243.0,1.0
1,24213,236.0,1.0
2,27116,243.0,1.0
3,28046,237.0,1.0
4,28787,243.0,1.0


Теперь нужно посчитать среднее количество баллов, которые получил каждый студент за экзамен(ы).

In [39]:
m_df = sc_ex. groupby ('id_student', as_index = False). agg ({'score' : 'mean'}). rename (columns = {'score': 'exam_score'})
m_df.head (5)

,id_student,exam_score
0,23698,80.0
1,24213,58.0
2,27116,96.0
3,28046,40.0
4,28787,44.0


In [40]:
rfm_df = rf_df. merge (m_df, on = 'id_student')
rfm_df.head(5)

# объединяем датафреймы со всеми метриками в один

,id_student,avg_time,completion_score,exam_score
0,23698,243.0,1.0,80.0
1,24213,236.0,1.0,58.0
2,27116,243.0,1.0,96.0
3,28046,237.0,1.0,40.0
4,28787,243.0,1.0,44.0


Теперь нужно разбить метрики на части по квантилям. Я выбрал децили, поскольку хотел захватить долю людей, не завершивших курс(ы). Так как таких людей в общей доле мало, то и делить пришлось на маленькие части.

In [41]:
quintiles = rfm_df[['avg_time', 'completion_score', 'exam_score']].quantile([.1, .2, .3, .4, .5, .6, .7, .8, .9]).to_dict()
quintiles

# создаём словарь с делением метрик на децили

{'avg_time': {0.1: 230.0,
  0.2: 231.0,
  0.3: 234.0,
  0.4: 237.0,
  0.5: 241.0,
  0.6: 242.0,
  0.7: 243.0,
  0.8: 243.5,
  0.9: 244.0},
 'completion_score': {0.1: 0.0,
  0.2: 1.0,
  0.3: 1.0,
  0.4: 1.0,
  0.5: 1.0,
  0.6: 1.0,
  0.7: 1.0,
  0.8: 1.0,
  0.9: 1.0},
 'exam_score': {0.1: 38.0,
  0.2: 46.200000000000045,
  0.3: 53.0,
  0.4: 60.0,
  0.5: 66.0,
  0.6: 72.0,
  0.7: 78.0,
  0.8: 84.0,
  0.9: 93.0}}

In [42]:
def r_score(x):
    if x <= quintiles['avg_time'][.1]:
        return 9
    elif x <= quintiles['avg_time'][.2]:
        return 8
    elif x <= quintiles['avg_time'][.3]:
        return 7
    elif x <= quintiles['avg_time'][.4]:
        return 6
    elif x <= quintiles['avg_time'][.5]:
        return 5
    elif x <= quintiles['avg_time'][.6]:
        return 4
    elif x <= quintiles['avg_time'][.7]:
        return 3
    elif x <= quintiles['avg_time'][.8]:
        return 2
    elif x <= quintiles['avg_time'][.9]:
        return 1
    else:
        return 0
    
def fm_score(x,c):
    if x <= quintiles[c][.1]:
        return 0
    elif x <= quintiles[c][.2]:
        return 1
    elif x <= quintiles[c][.3]:
        return 2
    elif x <= quintiles[c][.4]:
        return 3
    elif x <= quintiles[c][.5]:
        return 4
    elif x <= quintiles[c][.6]:
        return 5
    elif x <= quintiles[c][.7]:
        return 6
    elif x <= quintiles[c][.8]:
        return 7
    elif x <= quintiles[c][.9]:
        return 8
    else:
        return 9
    
# создаём функцию, которая присваивает каждому децилю число в зависимости от характера метрик

In [43]:
rfm_df['R'] = rfm_df['avg_time'].apply(lambda x: r_score(x))
rfm_df['F'] = rfm_df['completion_score'].apply(lambda x: fm_score(x, 'completion_score'))
rfm_df['M'] = rfm_df['exam_score'].apply(lambda x: fm_score(x, 'exam_score'))

# создаём колонки с оценкой метрик в соответствующем датафрейме 

In [44]:
rfm_df['RFM_Score'] = rfm_df['R'].map(str) + rfm_df['F'].map(str) + rfm_df['M'].map(str)
rfm_df

# склеиваем оценки метрик в общую колонку

,id_student,avg_time,completion_score,exam_score,R,F,M,RFM_Score
0,23698,243.0,1.0,80.0,3,1,7,317
1,24213,236.0,1.0,58.0,6,1,3,613
2,27116,243.0,1.0,96.0,3,1,9,319
3,28046,237.0,1.0,40.0,6,1,1,611
4,28787,243.0,1.0,44.0,3,1,1,311
...,...,...,...,...,...,...,...,...
4628,2694886,236.0,1.0,69.0,6,1,5,615
4629,2694933,230.0,1.0,73.0,9,1,6,916
4630,2695608,237.0,1.0,73.0,6,1,6,616
4631,2697181,230.0,1.0,80.0,9,1,7,917


Хотя сегментов получилось много, я придумал опредённый принцип деления. По завершаемости у нас всего две оценки (1 и 0), поскольку большая часть студентов в нашем датафрейме сдали экзамен, то функция присваивает им первую релевантную оценку (1). По остальным метрикам разброс оценрк от 1 до 9. Теперь подробнее о сегментации:

1) "Несправившихся" - студенты, затратившие много времени и не сдавшие экзамен;

2) "Слишком поторопившиеся" - студенты, затратившие мало времени и не сдавшие экзамен;

3) "Еле справившиеся" - студенты, затратившие много времени и сдавшие экзамен на минимальные баллы;

4) "Справившиеся ниже среднего" - студенты, затратившие много времени и сдавшие экзамен на баллы ниже средних;

5) "Основательные" - студенты, затратившие много времени и сдавшие экзамен на высокие баллы;

6) "Поверхностно освоившие" - студенты, затратившие мало времени и сдавшие экзамен на баллы ниже средних;

7) "Распылившиеся" - студенты, занимавшиеся на нескольких курсах, но успешно завершившие лишь один;

7) "Сбалансированные" - студенты, затратившие среднее количество времени и сдавшие экзамен на средние баллы;

8) "Быстрые" - студенты, затратившие мало времени и сдавшие экзамен на на средние баллы;

9) "Прилежные" - студенты, затратившие среднее количество времени и сдавшие экзамен на высокие баллы;

10) "Чемпионы" - студенты, затратившие мало времени и сдавшие экзамен на высокие баллы.

In [45]:
segt = {
    r'[0-4][0][0]': 'несправившиеся',
    r'[5-9][0][0]': 'слишком поторопившиеся',
    r'[0-4][1][1]': 'еле справившиеся',
    r'[0-4][1][2-5]': 'справившиеся ниже среднего',
    r'[0-4][1][6-9]': 'основательные',
    r'[5-9][1][1-4]': 'поверхностно освоившие',
    r'[1-9][1][0]'  : 'распылившиеся',
    r'[4-6][1][4-6]': 'сбалансированные',
    r'[7-9][1][4-6]': 'быстрые',
    r'[4-6][1][7-9]': 'прилежные',
    r'[7-9][1][7-9]': 'чемпионы'
}

rfm_df['Segment'] = rfm_df.RFM_Score
rfm_df['Segment'] = rfm_df['Segment'].replace(segt, regex=True)
rfm_df

# создаём сегментацию и заменяем ей колонку со склеенными значениями

,id_student,avg_time,completion_score,exam_score,R,F,M,RFM_Score,Segment
0,23698,243.0,1.0,80.0,3,1,7,317,основательные
1,24213,236.0,1.0,58.0,6,1,3,613,поверхностно освоившие
2,27116,243.0,1.0,96.0,3,1,9,319,основательные
3,28046,237.0,1.0,40.0,6,1,1,611,поверхностно освоившие
4,28787,243.0,1.0,44.0,3,1,1,311,еле справившиеся
...,...,...,...,...,...,...,...,...,...
4628,2694886,236.0,1.0,69.0,6,1,5,615,сбалансированные
4629,2694933,230.0,1.0,73.0,9,1,6,916,быстрые
4630,2695608,237.0,1.0,73.0,6,1,6,616,сбалансированные
4631,2697181,230.0,1.0,80.0,9,1,7,917,чемпионы


In [46]:
rfm_df.Segment. value_counts ()

# распределение студентов по сегментам

поверхностно освоившие        967
основательные                 881
справившиеся ниже среднего    880
чемпионы                      403
быстрые                       299
слишком поторопившиеся        269
несправившиеся                267
прилежные                     242
сбалансированные              241
еле справившиеся              178
распылившиеся                   6
Name: Segment, dtype: int64